## DATAFRAME OPERATIONS ON APACHE SPARK

In [1]:
#IMPORT LIBRARIES
!pip install findspark

import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F

spark = SparkSession \
.builder \
.master("local") \
.appName("PySpark Dataframe Homework") \
.getOrCreate()

In [5]:
# INDICATE FILE DIRECTORIES
data_path = "D:/retail_db"
output_path = "D:/pyspark_output_data"

In [10]:
# LIST THE RETAIL_DB
! ls -l D:/retail_db

total 9497
-rw-r--r-- 1 Unknown+User Unknown+Group    1074 Jun 20  2022 categories.csv
-rw-r--r-- 1 Unknown+User Unknown+Group  953847 Jun 20  2022 customers.csv
-rw-r--r-- 1 Unknown+User Unknown+Group      88 Jun 20  2022 departments.csv
-rw-r--r-- 1 Unknown+User Unknown+Group 5408988 Jun 20  2022 order_items.csv
-rw-r--r-- 1 Unknown+User Unknown+Group 2999990 Jun 20  2022 orders.csv
-rw-r--r-- 1 Unknown+User Unknown+Group  174240 Jun 20  2022 products.csv
-rw-r--r-- 1 Unknown+User Unknown+Group  174196 Jun 20  2022 products2.csv


### READ THE DATASETS WITH "inferSchema" OPTION , PRINT FIRST 3 OBSERVATIONS

In [11]:

categories = spark.read.option("inferSchema",True).csv(data_path+"/categories.csv", header=True)
categories.show(3)

+----------+--------------------+-------------------+
|categoryId|categoryDepartmentId|       categoryName|
+----------+--------------------+-------------------+
|         1|                   2|           Football|
|         2|                   2|             Soccer|
|         3|                   2|Baseball & Softball|
+----------+--------------------+-------------------+
only showing top 3 rows



In [12]:
categories.printSchema()

root
 |-- categoryId: integer (nullable = true)
 |-- categoryDepartmentId: integer (nullable = true)
 |-- categoryName: string (nullable = true)



In [15]:
products = spark.read.option("inferSchema",True).csv(data_path+"/products.csv", header=True)
products.show(3)

+---------+-----------------+--------------------+------------------+------------+--------------------+
|productId|productCategoryId|         productName|productDescription|productPrice|        productImage|
+---------+-----------------+--------------------+------------------+------------+--------------------+
|        1|                2|Quest Q64 10 FT. ...|              null|       59.98|http://images.acm...|
|        2|                2|Under Armour Men'...|              null|      129.99|http://images.acm...|
|        3|                2|Under Armour Men'...|              null|       89.99|http://images.acm...|
+---------+-----------------+--------------------+------------------+------------+--------------------+
only showing top 3 rows



In [16]:
products.printSchema()

root
 |-- productId: integer (nullable = true)
 |-- productCategoryId: integer (nullable = true)
 |-- productName: string (nullable = true)
 |-- productDescription: string (nullable = true)
 |-- productPrice: double (nullable = true)
 |-- productImage: string (nullable = true)



In [18]:
orders = spark.read.option("inferSchema",True).csv(data_path+"/orders.csv", header=True)
orders.show(3)

+-------+--------------------+---------------+---------------+
|orderId|           orderDate|orderCustomerId|    orderStatus|
+-------+--------------------+---------------+---------------+
|      1|2013-07-25 00:00:...|          11599|         CLOSED|
|      2|2013-07-25 00:00:...|            256|PENDING_PAYMENT|
|      3|2013-07-25 00:00:...|          12111|       COMPLETE|
+-------+--------------------+---------------+---------------+
only showing top 3 rows



In [19]:
orders.printSchema()

root
 |-- orderId: integer (nullable = true)
 |-- orderDate: string (nullable = true)
 |-- orderCustomerId: integer (nullable = true)
 |-- orderStatus: string (nullable = true)



In [20]:
order_items = spark.read.option("inferSchema",True).csv(data_path+"/order_items.csv", header=True)
order_items.show(3)

+-------------+----------------+------------------+-----------------+-----------------+---------------------+
|orderItemName|orderItemOrderId|orderItemProductId|orderItemQuantity|orderItemSubTotal|orderItemProductPrice|
+-------------+----------------+------------------+-----------------+-----------------+---------------------+
|            1|               1|               957|                1|           299.98|               299.98|
|            2|               2|              1073|                1|           199.99|               199.99|
|            3|               2|               502|                5|            250.0|                 50.0|
+-------------+----------------+------------------+-----------------+-----------------+---------------------+
only showing top 3 rows



In [21]:
order_items.printSchema()

root
 |-- orderItemName: integer (nullable = true)
 |-- orderItemOrderId: integer (nullable = true)
 |-- orderItemProductId: integer (nullable = true)
 |-- orderItemQuantity: integer (nullable = true)
 |-- orderItemSubTotal: double (nullable = true)
 |-- orderItemProductPrice: double (nullable = true)



In [22]:
## FIND OUT HOW MANY DISTINCT "orderItemOrderId" ON order_items table
order_items.select("orderItemOrderId").distinct().count()

57431

In [23]:
### FIND OUT HOW MANY ROWS ON order_items AND orders TABLES
orders.count()

68883

In [24]:
order_items.count()

172198

##  MOST CANCELLED PRODUCT GROUPS REGARDING TOTAL SALES ON DESCENDING ORDER
##  WRITE TO LOCAL STORAGE AS PARQUET  FILE FORMAT

In [25]:
# join the tables cat_products and products by selecting "productId","productName", "categoryName"
cat_products = products.join(categories, products['productCategoryId'] == categories['categoryId']) \
.select("productId","productName", "categoryName")

In [26]:
cat_products.limit(5).toPandas()

,productId,productName,categoryName
0,1,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,Soccer
1,2,Under Armour Men's Highlight MC Football Clea,Soccer
2,3,Under Armour Men's Renegade D Mid Football Cl,Soccer
3,4,Under Armour Men's Renegade D Mid Football Cl,Soccer
4,5,Riddell Youth Revolution Speed Custom Footbal,Soccer


In [27]:
# join orders AND order_items
orders_and_items = order_items.join(orders, order_items['orderItemOrderId'] == orders['orderId']) \
.select("orderId","orderItemProductId","orderItemSubTotal","orderStatus")
orders_and_items.limit(5).toPandas().head()

,orderId,orderItemProductId,orderItemSubTotal,orderStatus
0,1,957,299.98,CLOSED
1,2,1073,199.99,PENDING_PAYMENT
2,2,502,250.00,PENDING_PAYMENT
3,2,403,129.99,PENDING_PAYMENT
4,4,897,49.98,CLOSED


In [28]:
# Join recently created new tables above , and save as final_table
# Print first 5 as Pandas Dataframe
final_table = orders_and_items.join(cat_products, orders_and_items['orderItemProductId'] == cat_products['productId'])
final_table.limit(5).toPandas()

,orderId,orderItemProductId,orderItemSubTotal,orderStatus,productId,productName,categoryName
0,57760,858,199.99,PENDING_PAYMENT,858,GolfBuddy VT3 GPS Watch,Kids' Golf Clubs
1,57847,858,199.99,COMPLETE,858,GolfBuddy VT3 GPS Watch,Kids' Golf Clubs
2,58071,858,199.99,PENDING,858,GolfBuddy VT3 GPS Watch,Kids' Golf Clubs
3,58170,858,199.99,PENDING,858,GolfBuddy VT3 GPS Watch,Kids' Golf Clubs
4,58585,858,199.99,CANCELED,858,GolfBuddy VT3 GPS Watch,Kids' Golf Clubs


In [29]:
# FILTER THE FINAL_TABLE BASED ON CANCELLED ITEMS AND GROUP BY PRODUCT NAME THEN SUM IT BY SUBTOTALS
# ORDER BY DESCENDING ORDER REGARDING TOTAL PRICE
most_cancelled_products = final_table.filter("orderStatus == 'CANCELED'") \
.groupBy("productName").agg(F.sum("orderItemSubTotal").alias("Totalprice")) \
.orderBy(F.desc("TotalPrice"))
most_cancelled_products.limit(10).toPandas()

,productName,Totalprice
0,Field & Stream Sportsman 16 Gun Fire Safe,134393.28
1,Perfect Fitness Perfect Rip Deck,85785.70
2,Nike Men's Free 5.0+ Running Shoe,80691.93
3,Diamondback Women's Serene Classic Comfort Bi,80094.66
4,Pelican Sunstream 100 Kayak,66196.69
5,Nike Men's Dri-FIT Victory Golf Polo,65750.00
6,Nike Men's CJ Elite 2 TD Football Cleat,60705.33
7,O'Brien Men's Neoprene Life Vest,58126.74
8,Under Armour Girls' Toddler Spine Surge Runni,26153.46
9,LIJA Women's Eyelet Sleeveless Golf Polo,2145.00


In [ ]:
# WRITE TO LOCAL STORAGE AS PARQUET  FILE FORMAT
most_cancelled_products.coalesce(1) \
.write.mode("overwrite").parquet(output_path+"/most_cancelled_products_parquet")

In [31]:
most_cancelled_products.limit(5).toPandas()

,productName,Totalprice
0,Field & Stream Sportsman 16 Gun Fire Safe,134393.28
1,Perfect Fitness Perfect Rip Deck,85785.70
2,Nike Men's Free 5.0+ Running Shoe,80691.93
3,Diamondback Women's Serene Classic Comfort Bi,80094.66
4,Pelican Sunstream 100 Kayak,66196.69


## FIND OUT TOP 5 HIGHEST SALES WITH DATES

In [32]:
orders_and_items_date = order_items.join(orders, order_items['orderItemOrderId'] == orders['orderId']) \
.select("orderId","orderItemProductId","orderItemSubTotal","orderStatus","orderDate")
orders_and_items_date.limit(5).toPandas().head()

,orderId,orderItemProductId,orderItemSubTotal,orderStatus,orderDate
0,1,957,299.98,CLOSED,2013-07-25 00:00:00.0
1,2,1073,199.99,PENDING_PAYMENT,2013-07-25 00:00:00.0
2,2,502,250.00,PENDING_PAYMENT,2013-07-25 00:00:00.0
3,2,403,129.99,PENDING_PAYMENT,2013-07-25 00:00:00.0
4,4,897,49.98,CLOSED,2013-07-25 00:00:00.0


In [33]:
orders_and_items_date.printSchema()

root
 |-- orderId: integer (nullable = true)
 |-- orderItemProductId: integer (nullable = true)
 |-- orderItemSubTotal: double (nullable = true)
 |-- orderStatus: string (nullable = true)
 |-- orderDate: string (nullable = true)



In [34]:
# FIND OUT WHICH MONTHS AND YEARS ARE INDICATED IN DISTINCT COLUMNS
df_year_and_month = orders_and_items_date.withColumn("orderDate",
                                F.to_timestamp(F.col("orderDate"), "yyyy-MM-dd HH:mm:ss.S")) \
.withColumn("Year", F.year(F.col("orderDate"))) \
.withColumn("Month", F.month(F.col("orderDate")))


In [35]:
df_year_and_month.limit(5).toPandas()

,orderId,orderItemProductId,orderItemSubTotal,orderStatus,orderDate,Year,Month
0,1,957,299.98,CLOSED,2013-07-25,2013,7
1,2,1073,199.99,PENDING_PAYMENT,2013-07-25,2013,7
2,2,502,250.00,PENDING_PAYMENT,2013-07-25,2013,7
3,2,403,129.99,PENDING_PAYMENT,2013-07-25,2013,7
4,4,897,49.98,CLOSED,2013-07-25,2013,7


In [36]:
df_year_and_month.printSchema()

root
 |-- orderId: integer (nullable = true)
 |-- orderItemProductId: integer (nullable = true)
 |-- orderItemSubTotal: double (nullable = true)
 |-- orderStatus: string (nullable = true)
 |-- orderDate: timestamp (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)



### FIND OUT HIGHEST SALES WITH SUMS INDICATING YEAR AND MONTNS

In [37]:

df_year_and_month.filter("orderStatus not in ('CANCELED')") \
.groupBy('Year','Month').agg(F.sum("orderItemSubTotal").alias("Sum_SubTotal")) \
.orderBy(F.desc("Sum_SubTotal")) \
.limit(10).toPandas()

,Year,Month,Sum_SubTotal
0,2013,11,3105843.27
1,2014,1,2870834.18
2,2013,12,2869997.88
3,2013,9,2866553.33
4,2014,3,2805006.32
5,2013,8,2769236.03
6,2014,4,2758912.47
7,2014,2,2712838.58
8,2014,5,2695699.48
9,2014,6,2657013.04


In [38]:
spark.stop()